<a href="https://colab.research.google.com/github/inkyu0103/AIDetection/blob/main/%EA%B3%BC%EC%A0%9C%EC%96%B4%EB%A0%B5%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 많이 어렵 '조' 

- 조원 : **김인규 , 구다희**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/My\ Drive/데이터.zip

In [ ]:
# Modules install
!pip install pefile
!pip install numpy
!pip install sklearn
!pip install lightgbm
!pip install tqdm
!pip install pandas
!pip install graphviz
!pip install overload
!pip install torch
!pip install lief

In [ ]:
import os
import glob
import json
import pprint

import numpy as np

from lightgbm import LGBMClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.feature_extraction import FeatureHasher

In [ ]:
SEED = 41

def read_label_csv(path):
    label_table = dict()
    with open(path, "r",encoding='cp949') as f:
        for line in f.readlines()[1:]:
            fname, label = line.strip().split(",")
            label_table[fname] = int(label)
    return label_table

def read_json(path):
    with open(path, "r") as f:
        return json.load(f)

def load_model(**kwargs):
    if kwargs["model"] == "rf":
        return RandomForestClassifier(random_state=kwargs["random_state"], n_jobs=4)
    elif kwargs["model"] == "dt":
        return DecisionTreeClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lgb":
        return LGBMClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "svm":
        return SVC(random_state=kwargs["random_state"])
    elif kwargs["model"] == "lr":
        return LogisticRegression(random_state=kwargs["random_state"], n_jobs=-1)
    elif kwargs["model"] == "knn":
        return KNeighborsClassifier(n_jobs=-1)
    elif kwargs["model"] == "adaboost":
        return AdaBoostClassifier(random_state=kwargs["random_state"])
    elif kwargs["model"] == "mlp":
        return MLPClassifier(random_state=kwargs["random_state"])
    else:
        print("Unsupported Algorithm")
        return None
    

def train(X_train, y_train, model):
    clf = load_model(model=model, random_state=SEED)
    clf.fit(X_train, y_train)
    return clf


def evaluate(X_test, y_test, model):
    predict = model.predict(X_test)
    print("정확도 {} ".format(model.score(X_test, y_test)))


# 예측 데이터로부터 정답을 뽑아내는 함수 
def predict(x_test,model):
  predict = model.predict(x_test)
  print(predict)


In [ ]:
# 파일 이름
label_table_train = read_label_csv("/content/학습데이터_정답.csv")
label_table_test = read_label_csv("/content/검증데이터_정답.csv")


# PEMINER
- 새로운 특징 추가 x (188개 전부 사용)

---

# EMBER FEATURE

### 사용한 특징


- **histogram** --> 256차원 실수 벡터
- **stirngs**   --> 104차원 벡터
- **general**   --> 10차원 벡터
- **header**    --> 10차원 벡터
- **optional**  --> 14차원 벡터
- ~~ section   --> 6차원 벡터 ~~ (오류 때문에 삭제)
- **imports**   --> 1차원 벡터
- **exports**   --> 1차원 벡터 
- **datadirectories** --> 10차원 벡터


### 406개의 특징 벡터


---

# PESTUDIO

- **overview**
  - 모든 항목 사용

- **indicators**
  - hint 
  - xml-id
  - detail(length)
  - severity
  - text (length)

- **virustotal**
 - entropy
 - size

- **dos-header**
 - entropy
 - size
- **dos-stub**
 - entropy
 - size
- **overlay**
 - entropy
 - size

- **sections**
 - entropy





In [ ]:
class PeminerParser:
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
    
    def process_report(self):
        self.vector = [value for _, value in sorted(self.report.items(), key=lambda x: x[0])]
        return self.vector
    

class EmberParser:

    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
  #------------------------------------------------------------------------------------- ㅇ 256
    
    def get_histogram_info(self):
      histogram = np.array(self.report["histogram"])
      total = histogram.sum()
      vector = histogram / total
      

      return vector.tolist()
  
  #------------------------------------------------------------------------------------- ㅇ 104 
    def get_string_info(self):

      strings = self.report["strings"]
      hist_divisor = float(strings['printables']) if strings['printables'] > 0 else 1.0
      vector = [
          strings['numstrings'], 
          strings['avlength'], 
          strings['printables'],
          strings['entropy'], 
          strings['paths'], 
          strings['urls'],
          strings['registry'], 
          strings['MZ']
      ]
      vector += (np.asarray(strings['printabledist']) / hist_divisor).tolist()
      return vector
  #------------------------------------------------------------------------------------- ㅇ 10
    
    def get_general_file_info(self):

      try:
        general = self.report["general"]
        vector = np.hstack([
            general['size'], general['vsize'], general['has_debug'], general['exports'], general['imports'],
            general['has_relocations'], general['has_resources'], general['has_signature'], general['has_tls'],
            general['symbols']
        ]).tolist()


      # general 속성 없는 경우 10차원 벡터 추가
      except:
        vector = [-1]*10
      

      
      return vector
  #-------------------------------------------------------------------------------------  ㅇ10

    def get_header_info(self):
      header = self.report["header"]["coff"]
      # string list 는 배열의 길이로 결정

      self.vector=[ 
              header["timestamp"], len(header["machine"]),len(header["characteristics"])
      ]

      return self.vector

  #-------------------------------------------------------------------------------------  ㅇ 14 

    def get_optional_info(self):
      # optional이 없는 경우
      try : 
        optional = self.report["optional"]

        self.vector = [ 
                  len(optional["subsystem"]), len(optional["dll_characteristics"]), len(optional["magic"]), 
                  optional["major_image_version"], optional["minor_image_version"], optional["major_linker_version"], optional["minor_linker_version"],
                  optional["major_operating_system_version"], optional["minor_operating_system_version"], optional["major_subsystem_version"], optional["minor_subsystem_version"], optional["sizeof_code"],
                  optional["sizeof_headers"], optional["sizeof_heap_commit"]
        ]

      except : 
        self.vector  = [-1]*14

      return self.vector

  #------------------------------------------------------------------------------------- 6 (x)
    
    def get_section_info(self):
      section = self.report["section"]
      sections = section["sections"]
      vector =[
        len(section["entry"])
      ]


      av_name = 0
      av_size = 0
      av_entropy = 0
      av_vsize = 0
      av_props = 0 
      
      sec_len = len(sections)
      if sec_len == 0 :
        vector = [-1]*5

        return vector

      for i in sections :
        av_name += len(i["name"])
        av_size += i["size"]
        av_entropy += i["entropy"]
        av_vsize += i["vsize"]
        av_props += len(i["props"])

      tmp_vec = [av_name/sec_len, av_size/sec_len, av_entropy/sec_len,av_vsize/sec_len, av_props/sec_len]
      vector += tmp_vec

      
      return vector

  #------------------------------------------------------------------------------------- ㅇ1

    def get_imports_info(self):
      '''
        imports 부분이 dictionary라 몇 개가 들어올지 모르기 때문에, key의 개수로 하도록 할게요
        그리고 키 값도 다를 수 있으니... key만 뽑아서 할 것.

      '''

      imports = self.report["imports"]
      import_key_list = len(imports.keys())
      
      
      vector = [import_key_list]
      return vector

  #------------------------------------------------------------------------------------- ㅇ 1
    
    def get_exports_info(self):
      '''
      export부분의 원소가 어떤 type인지 모르기 때문에, export도 len으로 맞춰주었습니다.

      만약 string으로 고정된다면, string length의 평균을 기준으로 해보려고 했어요.
      '''

      exports = self.report["exports"]
      vector = [len(exports)]

      return vector

  #-------------------------------------------------------------------------------------
  
    def get_datadirectories_info(self):
      vector = []
      '''
        name , size , virtual_addr 이 세부분으로 이루어져 있으니 평균을 내어 구합니다.
      '''
      datadirectories = self.report["datadirectories"]
      
      av_name = 0
      av_size = 0 
      av_virtual_addr = 0 

      length = len(datadirectories)
      
      if length == 0 :
        vector =[-1]*3
        return vector
      
      for i in datadirectories:
        av_name += len(i["name"])
        av_size += i["size"]
        av_virtual_addr += i["virtual_address"]

      tmp_vec = [av_name/length , av_size / length, av_virtual_addr/length]

      vector += tmp_vec
      return vector
  #-------------------------------------------------------------------------------------

    def process_report(self):
      vector = []
      vector += self.get_histogram_info()  # O
      vector += self.get_string_info() # O
      vector += self.get_general_file_info() # O
      vector += self.get_header_info() # O
      vector += self.get_optional_info() # ㅇ
      #vector += self.get_section_info()  x      
      vector += self.get_optional_info() # ㅇ
      vector += self.get_imports_info() # ㅇ
      vector += self.get_exports_info() # ㅇ
      vector += self.get_datadirectories_info() # ㅇ  

      return vector


############################################################################################

class PestudioParser:        
    def __init__(self, path):
        self.report = read_json(path)
        self.vector = []
  #------------------------------------------------------------------------------------- 25

    def get_overview_info (self):
      
      v = []
      overview = self.report["image"]["overview"]

      #overview key list
      overview_key_list = list(self.report["image"]["overview"].keys())

      int_candidate = ["cpu","size","size-without-overlay","entropy"]
      for item in overview_key_list : 
        if item in int_candidate:
          if overview[item] != "n/a":
            v.append(float(overview[item]))
          else:
            v.append(-1)
        else:
          if overview[item] == None:
            v.append(-1)
          elif len(overview[item]) >3 :
            v.append(len(overview[item]))
          else:
            v.append(-1)
      
      return v
          
  #------------------------------------------------------------------------------------- 5차원
    
    def get_indicators_info (self):
      vector = []
      indicators = self.report["image"]["indicators"]
      # hint
      vector.append(len(indicators["@hint"])) if len(indicators["@hint"]) > 3 else vector.append(-1)
    
      #indicator (indicator가 없는 파일도 존재)
      try : 
        indicator = indicators["indicator"]

        l = len(indicator)
        av_xml = 0
        av_detail = 0  # (length로)
        av_severity = 0
        av_text = 0 #(length로)

      
        for i in indicator:
          av_xml += int(i["@xml-id"])
          av_detail = len(i["@detail"])
          av_severity = int(i["@severity"])
          av_text = len(i["#text"])

        if l != 0 :
          vector.append(av_xml/l)
          vector.append(av_detail/l)
          vector.append(av_severity/l)
          vector.append(av_text/l)

        if l == 0:
          vector += [-1,-1,-1,-1]


      except : 
        #indicator가 없는 경우
        vector += [-1,-1,-1,-1]

      return vector

        
  #------------------------------------------------------------------------------------- 2차원

    # 유무로 판단
    def get_virustotal_info(self):
      try:
        vector = []
        virustotal = self.report["image"]["virustotal"]


        detection = virustotal["@detection"]
        engine = virustotal["@engine"]

        vector += int(detection)
        vector += int(engine)

      except:
        vector +=[-1,-1]

      return vector

  #------------------------------------------------------------------------------------- 2차원

    def get_dos_header_info(self):
      vector = []
      try:
        dos_header = self.report["image"]["dos-header"]

      except:
        vector += [-1,-1]
        return vector
      
      try:
        entropy = dos_header["entropy"]
        vector.append(int(entropy))
      except:
        vector.append(-1)
      try:
        size = dos_header["size"]
        vector.append(int(size))
      except:
        vector.append(-1)

      

      return vector



  #------------------------------------------------------------------------------------- 2차원 
    def get_dos_stub_info(self):
      vector = []
      try:
        dos_stub = self.report["image"]["dos-stub"]
      except:
        vector += [-1,-1]
        return vector
      try:
        entropy = dos_stub["entropy"]
        vector.append(int(entropy))
      except:
        vector.append(-1)
      try:
        size = dos_stub["size"]
        vector.append(int(size))
      except:
        vector.append(-1)

      return vector
        
    #------------------------------------------------------------------------------------- 1차원 

    def get_sections_info(self):
      vector =[]
      try:
        sections = self.report["image"]["sections"]
      except:
        vector.append(-1)
        return vector

      try:
        av_entropy = 0
        section = sections["section"]
        for i in section:
          av_entropy += float(i["@entropy"])
        
        vector.append(av_entropy/len(section))

      except:
        vector.append(-1)

      return vector


    #------------------------------------------------------------------------------------- 2차원 

    def get_overlay_info(self):
      vector = []
      try:
        overlay = self.report["image"]["overlay"]
      except:
        vector += [-1,-1]
        return vector

      try : 
        entropy = float(overlay["entropy"])
        vector.append(entropy)
      except:
        vector.append(-1)
      
      try:
        size = float(overlay["size"])
        vector.append(size)

      except:
        vector.append(-1)

      return vector



      



  
  #-------------------------------------------------------------------------------------
    
    def process_report(self):
      vector = [] 
      vector += self.get_overview_info()  #
      vector += self.get_indicators_info() #
      vector += self.get_virustotal_info() #
      vector += self.get_dos_header_info() #
      vector += self.get_dos_stub_info() #
      vector += self.get_sections_info() #
      vector += self.get_overlay_info() #


      return vector



In [ ]:
pestudio_test_file =  []

# pestudio_test_file (.json 빠진 파일 이름만 있는 리스트)
for i in  glob.glob("/content/PESTUDIO/테스트데이터/*.json"):
  pestudio_test_file.append(os.path.basename(i).split(".")[0])



other_test_file = []
# other_test_file (.json 빠진 파일 이름만 있는 리스트)
for i in  glob.glob("/content/PEMINER/테스트데이터/*.json"):
  other_test_file.append(os.path.basename(i).split(".")[0])


real_test_file = list(set(pestudio_test_file).intersection(other_test_file))

print(len(real_test_file))




9809


In [ ]:

###################### TRAIN DATA ####################
# fname_list =>  모든 학습 파일이름(확장자 제외) 
fname_train_list = list(label_table_train)


# PESTUDIO TRAIN DATA
pestudio_train_list=[]
for path in glob.glob("/content/PESTUDIO/학습데이터/*.json"):
  pestudio_train_list.append(os.path.basename(path).split(".")[0])

#real_train_list : 겹치는 파일 (19940 개)
real_train_list = list(set(fname_train_list).intersection(pestudio_train_list))


###################### VALIDATION DATA ####################

# fname_list (.json 빠진 파일 이름만 있는 리스트)
fname_validate_list = list(label_table_test)

# PESTUDIO VALIDATION DATA
pestudio_validate_list=[]
for path in glob.glob("/content/PESTUDIO/검증데이터/*.json"):
  pestudio_validate_list.append(os.path.basename(path).split(".")[0])

#real_validate_list : 겹치는 파일 (9798 개)
real_validate_list = list(set(fname_validate_list).intersection(pestudio_validate_list))


###################### TEST DATA ####################


pestudio_test_file =  []

# pestudio_test_file (.json 빠진 파일 이름만 있는 리스트)
for i in  glob.glob("/content/PESTUDIO/테스트데이터/*.json"):
  pestudio_test_file.append(os.path.basename(i).split(".")[0])



other_test_file = []
# other_test_file (.json 빠진 파일 이름만 있는 리스트)
for i in  glob.glob("/content/PEMINER/테스트데이터/*.json"):
  other_test_file.append(os.path.basename(i).split(".")[0])


#9809 개 
real_test_file = list(set(pestudio_test_file).intersection(other_test_file))


################### TEST DATA2  (나머지) #######################

resi_test_file = list(set(other_test_file)- set(pestudio_test_file))



# 188 / 406 / 39



# x_train : 훈련집합 특징 벡터
# y_train : 훈련집합 정답
tr = 0
x_train, y_train = [], []
for fname in fname_train_list:  
  print("tr {} ".format(tr))
  feature_vector = []
  # label_table --> dictionary { file_name : answer }
  label = label_table_train[fname] 

  for data in ["PEMINER","EMBER","PESTUDIO"]:
      path = f"/content/{data}/학습데이터/{fname}.json"
      if data == "PESTUDIO" and fname in real_train_list:
        feature_vector += PestudioParser(path).process_report()
      elif data =="PESTUDIO":
        feature_vector += = [-1]*39
      elif data == "PEMINER":
        feature_vector += PeminerParser(path).process_report()
      elif data =="EMBER":
        feature_vector += EmberParser(path).process_report()
  x_train.append(feature_vector)
  y_train.append(label)
  tr += 1

print("next")

x_val,y_val = [],[]
va = 0
# 테스트 집합 파일 리스트 ()
fname_list = list(label_table_test)

# pestudio 테스트 집합 이름 (루트~  파일명)
pestudio_test_list = glob.glob("/content/PESTUDIO/검증데이터/*.json")


for fname in fname_validate_list:
  print("검증 data {}".format(va))

  feature_vector = []
  label = label_table_test[fname]
  for data in ["PEMINER","EMBER","PESTUDIO"]:
    path = f"/content/{data}/검증데이터/{fname}.json"
    if data == "PESTUDIO" and real_validate_list:
      feature_vector += PestudioParser(path).process_report()      
    elif data =="PESTUDIO":
      feature_vector += [-1]*39
    elif data == "PEMINER":
      feature_vector += PeminerParser(path).process_report()
      
    elif data == "EMBER":
      feature_vector += EmberParser(path).process_report()
      
  x_val.append(feature_vector)
  y_val.append(label)
  va+=1


x_test,y_test= [],[]
t  = 0 
for fname in other_test_file:
  print("test data {}".format(t))
  feature_vector = []
  for data in ["PEMINER","EMBER","PESTUDIO"]:
      path = f"/content/{data}/테스트데이터/{fname}.json"

      if data == "PESTUDIO" and fname in real_test_file:
        feature_vector += PestudioParser(path).process_report()

      elif data == "PESTUDIO": 
        feature_vector += [-1]*39      

      elif data == "PEMINER":
        feature_vector += PeminerParser(path).process_report()
        
      elif data == "EMBER":
        feature_vector += EmberParser(path).process_report()

  x_test.append(feature_vector)
  t+=1

  #633개 특징


In [ ]:
# 학습
models = []
for model in ["rf", "lgb"]:
    clf = train(x_train, y_train, model)
    models.append(clf)

# 검증

for model in models:
    evaluate(x_val, y_val, model)



정확도 0.9454990814451929 
정확도 0.9526433966115534 


In [ ]:
def ensemble_result(X, y, models):
    predicts = []
    for model in models:
        prob = [result for _, result in model.predict_proba(X)]
        predicts.append(prob)
    
    predict = np.mean(predicts, axis=0)
    predict = [1 if x >= 0.5 else 0 for x in predict]
    
    print("정확도", accuracy_score(y, predict))




    '''
    학습과정에서 pestudio에 없는 파일은 학습시키지 않았을 때, rf : 0.9455 ... / lgb : 0.9526 / 앙상블 : 0.9539 ... 


    '''

In [ ]:
ensemble_result(x_val,y_val,models)

정확도 0.9538681363543581


In [ ]:
def predict(x_test,models):
  for model in models:
    predict = model.predict(x_test)
  return predict 

# 테스트 데이터에 대한 예측 값
predict_result = predict(x_test,models)


In [ ]:
# csv 파일 쓰기

import csv

data_list = []
f = open('predict.csv','w')
csv_writer = csv.writer(f)
csv_writer.writerow(['file','predict'])
for i in range(10000):
  csv_writer.writerow([other_test_file[i],predict_result[i]])
f.close()